## Theoretical Background  
### Sensor Fusion and Spoofing in Autonomous Systems  
Autonomous vehicles rely on **multi-sensor fusion**, combining data from complementary sensors such as cameras and LiDARs to perceive their environment accurately. Each sensor type has strengths and weaknesses — for example, cameras provide rich color and texture information, while LiDAR offers precise depth measurements. Fusing their outputs can significantly improve perception robustness and decision-making.  

However, autonomous systems are also vulnerable to **sensor spoofing attacks**, where false or corrupted data is injected into sensor streams. These attacks can degrade model accuracy and lead to dangerous misperceptions. Understanding how spoofing impacts sensor fusion systems is crucial for designing resilient AI-based perception pipelines.  

---

## Problem Statement  
The purpose of this experiment is to analyze the **impact of sensor spoofing** on multi-sensor data fusion. The system models two sensors — a simulated camera and LiDAR — generating synthetic data with controlled correlations. By introducing artificial spoofing noise to one sensor, the experiment compares classification accuracy across individual and fused sensor models.  

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## Methodology  
### Synthetic Data Generation  
The function `make_synthetic_data()` creates artificial sensor features and corresponding binary labels.  
- A **latent variable** represents the true environmental state.  
- Both **camera** and **LiDAR** features are generated by adding small Gaussian noise to this latent representation.  
- The **label** is determined by whether the sum of the latent features exceeds a threshold (representing a simple binary classification problem).  

### Sensor Spoofing Simulation  
The `spoof_sensor()` function adds **strong Gaussian noise** to a sensor’s feature set, simulating a spoofing or tampering event that corrupts the sensor’s reliability.

In [ ]:
def make_synthetic_data(n_samples=1000, dim=5, noise=0.1, random_seed=0):
    """
    Create synthetic features:
    cam_features = true latent + small noise
    lidar_features = true latent + small noise
    label = binary based on latent > threshold
    """
    np.random.seed(random_seed)
    latent = np.random.randn(n_samples, dim)
    cam = latent + noise * np.random.randn(n_samples, dim)
    lidar = latent + noise * np.random.randn(n_samples, dim)
    # Label: sum(latent) > 0 is class 1
    y = (latent.sum(axis=1) > 0).astype(int)
    return cam, lidar, y

def spoof_sensor(sensor_feats, strength=1.0):
    """
    Add large noise to simulate spoofing / tampering
    """
    n, d = sensor_feats.shape
    noise = strength * np.random.randn(n, d)
    return sensor_feats + noise

### Feature Fusion  
The fusion strategy (`fuse_features()`) concatenates camera and LiDAR features, representing a simple yet effective early fusion approach for combining multi-modal information.  


In [ ]:
def fuse_features(cam, lidar):
    """
    Simple concatenation fusion
    """
    return np.concatenate([cam, lidar], axis=1)

### Training and Evaluation  
Three **Logistic Regression** classifiers are trained and tested:
1. **Camera-only model** – trained using only camera features.  
2. **LiDAR-only model** – trained using only LiDAR features.  
3. **Fused model** – trained on combined camera and LiDAR features.  

Performance is measured using **accuracy** on both normal and spoofed test sets to assess the resilience of fusion strategies.  

---

## Experimental Design  
Two conditions are evaluated:
- **Normal condition:** both camera and LiDAR sensors operate normally.  
- **Spoofed condition:** the camera sensor is compromised using high-intensity noise (`strength=2.0`).  

By comparing classification accuracies across these conditions, the experiment quantifies how **sensor spoofing degrades performance** and whether **sensor fusion** can mitigate its impact.  

In [ ]:
def train_and_eval(cam_train, lidar_train, y_train,
                   cam_test, lidar_test, y_test):
    # (1) camera-only
    clf_cam = LogisticRegression(max_iter=200)
    clf_cam.fit(cam_train, y_train)
    acc_cam = accuracy_score(y_test, clf_cam.predict(cam_test))
    # (2) lidar-only
    clf_lidar = LogisticRegression(max_iter=200)
    clf_lidar.fit(lidar_train, y_train)
    acc_lidar = accuracy_score(y_test, clf_lidar.predict(lidar_test))
    # (3) fused
    clf_fuse = LogisticRegression(max_iter=200)
    clf_fuse.fit(fuse_features(cam_train, lidar_train), y_train)
    acc_fuse = accuracy_score(y_test, clf_fuse.predict(fuse_features(cam_test, lidar_test)))
    return acc_cam, acc_lidar, acc_fuse


In [ ]:
cam, lidar, y = make_synthetic_data(n_samples=2000, dim=8, noise=0.1)
# Introduce spoofing on camera sensor in test set
cam_train, cam_test, lidar_train, lidar_test, y_train, y_test = train_test_split(
    cam, lidar, y, test_size=0.3, random_state=0)
# Spoof camera in test set
cam_test_spoof = spoof_sensor(cam_test, strength=2.0)

# Evaluate performance when camera is spoofed
print("When camera sensor is normal (no spoof):")
accs_normal = train_and_eval(cam_train, lidar_train, y_train,
                              cam_test, lidar_test, y_test)
print("Cam-only, Lidar-only, Fusion:", accs_normal)

print("When camera sensor is spoofed:")
accs_spoof = train_and_eval(cam_train, lidar_train, y_train,
                            cam_test_spoof, lidar_test, y_test)
print("Cam-only, Lidar-only, Fusion:", accs_spoof)

## Observations and Computational Challenge  
While simple fusion improves robustness in clean conditions, it remains partially susceptible when one sensor is heavily spoofed. This highlights a fundamental challenge in autonomous systems: **balancing fusion benefits with sensor reliability**. Real-world deployment would require adaptive fusion mechanisms capable of dynamically weighting sensor trustworthiness under adversarial or noisy conditions.